# Figures for attribution presentation

## pdf shift plot

### Gaussian

In [210]:
library(shape)

x <- seq(24,35,0.1)

mean_f <- 30.562; sd_f <- 1.343
mean_cf <- mean_f - 1.2; sd_cf <- sd_f - 0.1

obs <- 32 #qnorm(0.8, mean_f, sd_f)
xx <- seq(obs,35,0.01)

png("pdf_gaussian_shiftscale.png", height = 480, width = 480); {

    # set plot parameters (scale text by 1.5x, set outer margins to remove whitespace)
    par(cex = 1.5, mar = c(4,2,1,1))
    
    # create an empty plot - will probably need to change height of y-axis via second value in ylim
    plot(0, type = "n", xaxs = "i", yaxs = "i", bty = "n", yaxt = "n", xlim = c(min(x) - 0.2, max(x)), ylim = c(0,0.425), xlab = "", ylab = "")
    ymax <- par("usr")[4]*0.925

    # axes & axis labels
    mtext(expression("Temperature ("*degree*"C)"), side = 1, line = 2.5, cex = 1.5)
    mtext("Likelihood", side = 2, cex = 1.5)
    Arrows(min(x),0,min(x),ymax, lwd = 3)
    Arrows(min(x),0,max(x),0, lwd = 3)

    # add lines showing counterfactual & factual densities
    lines(x, dnorm(x, mean = mean_cf, sd = sd_cf), type = "l", lwd = 3, col = "blue")
    lines(x, dnorm(x, mean = mean_f, sd = sd_f), type = "l", lwd = 3, col = "red3")
    abline(h = 0, lwd = 3)

    # add shaded polygons - currently non-overlapping 
    polygon(c(xx, rev(xx)), c(rep(0,length(xx)), rev(dnorm(xx, mean = mean_cf, sd = sd_cf))), col = adjustcolor("blue", alpha = 0.3), border = NA)
    polygon(c(xx, rev(xx)), c(dnorm(xx, mean = mean_cf, sd = sd_cf), rev(dnorm(xx, mean = mean_f, sd = sd_f))), col = adjustcolor("red", alpha = 0.3), border = NA)

    # line marking observed temperature
    lines(rep(obs,2), c(0,ymax), col = "dimgrey", lty = 2, lwd = 3)
    text(obs, ymax, "Observed", col = "dimgrey", pos = 3)
    

    # add labels (will need to play around with x & y coordinates to get best location)
    text(28.3,.225, "Cooler\nclimate", pos = 2, col = "blue")
    # text(26.5,.2, expression(cooler), pos = 2, col = "blue")

    text(30.6,.33, "Current\nclimate", col = "red3")

    text(33.3,.07, "P1", col = "red3")
    text(33,.02, "P0", col = "blue")

}; dev.off()

png 
  2

### GEV

In [226]:
library(shape)

x <- seq(25,36,0.1)

mean_f <- 30.562; sd_f <- 1.343; xi <- -0.2
mean_cf <- mean_f - 1.2; sd_cf <- sd_f - 0.1

obs <- 32
xx <- seq(obs,max(x),0.01)

png("pdf_gev_shiftscale.png", height = 480, width = 480); {

    # set plot parameters (scale text by 1.5x, set outer margins to remove whitespace)
    par(cex = 1.5, mar = c(4,2,1,1))
    
    # create an empty plot - will probably need to change height of y-axis via second value in ylim
    plot(0, type = "n", xaxs = "i", yaxs = "i", bty = "n", yaxt = "n", xlim = c(min(x) - 0.2, max(x)), ylim = c(0,0.465), xlab = "", ylab = "")
    ymax <- par("usr")[4]*0.925

    # axes & axis labels
    mtext(expression("Temperature ("*degree*"C)"), side = 1, line = 2.5, cex = 1.5)
    mtext("Likelihood", side = 2, cex = 1.5)
    Arrows(min(x),0,min(x),ymax, lwd = 3)
    Arrows(min(x),0,max(x),0, lwd = 3)

    # add lines showing counterfactual & factual densities
    lines(x, devd(x, loc = mean_cf, scale = sd_cf, shape = xi), type = "l", lwd = 3, col = "blue")
    lines(x, devd(x, loc = mean_f, scale = sd_f, shape = xi), type = "l", lwd = 3, col = "red3")
    abline(h = 0, lwd = 3)

    # add shaded polygons - currently non-overlapping 
    polygon(c(xx, rev(xx)), c(rep(0,length(xx)), rev(devd(xx, loc = mean_cf, scale = sd_cf, shape = xi))), col = adjustcolor("blue", alpha = 0.3), border = NA)
    polygon(c(xx, rev(xx)), c(devd(xx, loc = mean_cf, scale = sd_cf, shape = xi), rev(devd(xx, loc = mean_f, scale = sd_f, shape = xi))), col = adjustcolor("red", alpha = 0.3), border = NA)

    # line marking observed temperature
    lines(rep(obs,2), c(0,ymax), col = "dimgrey", lty = 2, lwd = 3)
    text(obs, ymax, "Observed", col = "dimgrey", pos = 3)
    

    # add labels (will need to play around with x & y coordinates to get best location)
    text(28.3,.225, "Cooler\nclimate", pos = 2, col = "blue")
    # text(26.5,.2, expression(cooler), pos = 2, col = "blue")

    text(30.6,.33, "Current\nclimate", col = "red3")

    text(33,.07, "P1", col = "red3")
    text(32.4,.02, "P0", col = "blue")

}; dev.off()

png 
  2

# CET time series plots

In [9]:
suppressMessages(library(rwwa))
library(shape)

df <- merge(read.csv("cet_tx1x.csv", col.names = c("year", "tmax")), read.csv("gmst.csv", col.names = c("year", "gmst")))
df$gmst <- df$gmst - df$gmst[df$year == 2022]

mdl <- fit_ns("gev", "shift", df, "tmax","gmst", lower = F, ev_year = 2022)
y_pi <- ns_pars(mdl, fixed_cov = data.frame("gmst" = -1.2))$loc
y_2022 <- ns_pars(mdl, fixed_cov = data.frame("gmst" = 0))$loc

loess_fit <- fitted(loess(tmax ~ gmst, df))

erl_10 <- eff_return_level(mdl, 10)[order(df$gmst)]

In [198]:
png("cet-timeseries-1.png", height = 480, width = 480*2); {
    plot(df[,c("year", "tmax")], type = "S", lwd = 2, xlab = "", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series")
    points(df[df$year == 2022,c("year", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
}; dev.off()

png 
  2

In [199]:
png("cet-timeseries-2.png", height = 480, width = 480*2); {
    plot(df[,c("year", "tmax")], type = "S", lwd = 2, xlab = "", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series")
    points(df[df$year == 2022,c("year", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$year, fitted(loess(tmax ~ year, df)), col = "forestgreen", lw = 2)
}; dev.off()

png 
  2

In [200]:
png("cet-timeseries-3_vs-gmst.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
}; dev.off()

png 
  2

In [201]:
png("cet-timeseries-4_vs-gmst-loess.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
}; dev.off()

png 
  2

In [ ]:
png("cet-timeseries-5_linear-model.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
}; dev.off()

In [3]:
png("cet-timeseries-4_linear-model-only.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
}; dev.off()

png 
  2

In [203]:
png("cet-timeseries-6_gmst-change.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    Arrows(-1.2,y_pi,0,y_pi, col = "orange", lw = 2, arr.adj = 1, arr.type = "curved")
    text(-0.4, y_pi-1, "GMST has increased by 1.2C", col = "orange", font = 2)

    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
}; dev.off()

png 
  2

In [204]:
png("cet-timeseries-7_tmax-change.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    Arrows(-1.2,y_pi,0,y_pi, col = "orange", lw = 2, arr.adj = 1, arr.type = "curved")
    text(-0.4, y_pi-1, "GMST has increased by 1.2C", col = "orange", font = 2)
    Arrows(0,y_pi,0,y_2022, col = "orange", lw = 2, arr.adj = 1, arr.type = "curved")
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    text(0.02, y_2022-2.5, "Average\ntmax\n increases\nby 3C", col = "orange", font = 2, adj = 0)

}; dev.off()

png 
  2

In [206]:
png("cet-timeseries-8_return-levels.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    lines(df$gmst[order(df$gmst)], erl_10, col = "darkred", lw = 2, lty = 2)
    text(0.05, erl_10[length(erl_10)], "1-in-10-year\nevent", adj = 0, col = "darkred", font = 2)
}; dev.off()

png 
  2

In [13]:
png("cet-timeseries-9_event-probability.png", height = 480, width = 480*2); {
    plot(df[,c("gmst", "tmax")], pch = 20, lwd = 2, xlab = "Global mean surface temperature (GMST) anomaly vs 2022", ylab = "Maximum temperature (degC)",
        main = "Central England Temperature series", xlim = c(-1.45,0.25))
    points(df[df$year == 2022,c("gmst", "tmax")], pch = 21, bg = "magenta", cex = 1.5)
    lines(df$gmst, loess_fit, col = "forestgreen", lw = 2)
    lines(df$gmst[order(df$gmst)], ns_pars(mdl)$loc[order(df$gmst)], col = "darkred", lw = 2)
    lines(df$gmst[order(df$gmst)], erl_10, col = "darkred", lw = 2, lty = 2)
    text(0.05, erl_10[length(erl_10)], "1-in-10-year\nevent", adj = 0, col = "darkred", font = 2)
    text(0.035, mdl$ev-0.2, "1-in-800-year\nevent", adj = 0, col = "darkred", font = 2)
}; dev.off()

png 
  2

In [5]:
return_period(m,dl)

[1]   1.009454   1.997641   3.882118   1.044982   1.071944   5.637932
  [7]   4.297893   2.147384   3.228017   1.625324   1.128186   1.025579
 [13]   1.353753   1.304706   5.088151   2.306708   1.781021   1.887352
 [19]   3.569123   1.854295   2.999584   4.124400   5.488564   2.261918
 [25]   1.204572   1.885716   1.503303  27.629302   1.126020   2.567891
 [31]   1.836357   1.330686  26.375658   6.905609   1.423405   3.710757
 [37]   1.229916   1.488344   3.129191   1.825320   2.230181   1.159694
 [43]  10.607414   1.926192  19.404446   3.750185   3.371288   3.677792
 [49]   1.239884   1.436283   3.455057  12.022592   1.014502   9.864370
 [55]   3.878262   4.240488   2.959436   1.982872   2.000673   1.357976
 [61]   2.564653   2.233621   4.242420   4.568865   8.758683   2.890913
 [67]   1.248492   1.712623   6.370166  20.628659   8.208330   6.874919
 [73]   1.155939   3.483075   5.655126   1.461259   2.561271   1.035339
 [79]   2.673283   1.116072   2.914888   2.155858   4.667968   1.002986
 [85]   1.507637   1.336149   1.323375   1.177400   1.713237   1.376615
 [91]   2.431131   6.743823   1.577863   1.158824   2.738877   1.075413
 [97]  24.944988  78.301265   1.392076   1.274397   1.528026   1.575439
[103]   1.094938   1.605472   2.904812   4.432658   1.278505   1.924899
[109]   1.317499   1.524062   3.026524  40.580885   1.373068   1.778024
[115]   1.035684   1.553195  12.680339   2.661162   4.184364   1.122978
[121]   2.684010   2.224579   1.841149   1.399185  15.705136   1.826811
[127]   2.824560  16.699719   1.115491   1.117061   1.449657   1.033335
[133]   1.285761   1.065896   2.244860   1.196019   9.791104   2.959044
[139]   1.727097   2.682336  23.494544  11.128708   1.521476 814.425161
[145]   1.390145   1.061544